<a href="https://colab.research.google.com/github/abs711/nanoGPT/blob/master/nanoGPT_from_scratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
    !wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2023-11-29 21:51:00--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.109.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt’

input.txt           100%[===================>]   1.06M  --.-KB/s    in 0.06s   

2023-11-29 21:51:00 (18.4 MB/s) - ‘input.txt’ saved [1115394/1115394]



In [ ]:
with open('input.txt', 'r', encoding='utf-8') as f:
  text = f.read()

In [ ]:
f"Total document length: {len(text)}"

'Total document length: 1115394'

In [ ]:
#Unique characters in the dataset
chars = sorted(list(set(text)))
vocab_size = len(chars)

print(''.join(chars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [ ]:
#Tokenization

stoi = {ch:i for i, ch in enumerate(chars)}

itos = {i:ch for i, ch in enumerate(chars)}

encode = lambda s: [stoi[c] for c in s]
decode = lambda l: ''.join([itos[i] for i in l])

print(encode("Please encode"))

print(decode(encode("please decode")))

[28, 50, 43, 39, 57, 43, 1, 43, 52, 41, 53, 42, 43]
please decode


In [ ]:
import torch

data = torch.tensor(encode(text), dtype=torch.long)
f"{data.shape}, {data.dtype}"

'torch.Size([1115394]), torch.int64'

In [ ]:
n = int(0.9*len(data))

train_data = data[:n]
test_data = data[n:]

In [ ]:
torch.manual_seed(1337)
batch_size = 32
block_size = 8 #context_length
n_embd = 32 #number of embedding dimensions
learning_rate = 1e-3
max_iters = 5000
dropout = 0.5

In [ ]:
class BatchNorm1D: # Normalize Columns
  def __init__(self, dim, eps=1e-5, momentum=0.1):
    self.eps = eps
    self.momentum = momentum
    self.training = True
    self.gamma = torch.ones(dim)
    self.beta = torch.zeros(dim)

    self.running_mean = torch.zeros(dim)
    self.runnning_var = torch.ones(dim)

  def __call__(self, x):
    if self.training:
      xmean = x.mean(0, keepdim=True)
      xvar = x.var(0, keepdim=True)
    else:
      xmean = self.running_mean
      xvar = self.runnning_var
    xhat = (x - xmean) / torch.sqrt(xvar + self.eps)
    self.out = self.gamma * xhat + self.beta

    if self.training:
      with torch.no_grad():
        self.running_mean = (1 - self.momentum)* self.running_mean + self.momentum * xmean
        self.running_var = (1 - self.momentum)* self.runnning_var + self.momentum * xvar

    return self.out

  def parameters(self):
    return [self.gamma, self.beta]


class LayerNorm1D: # Normalize Rows
  def __init__(self, dim, eps=1e-5):
    self.eps = eps
    self.gamma = torch.ones(dim)
    self.beta = torch.zeros(dim)
  def __call__(self, x):
    xmean = x.mean(1, keepdim=True)
    xvar = x.var(1, keepdim=True)
    xhat = (x - xmean) / torch.sqrt(xvar + self.eps)
    self.out = self.gamma * xhat + self.beta

    return self.out

  def parameters(self):
    return [self.gamma, self.beta]

In [ ]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

class Head(nn.Module):
  """a single head of self-attention"""
  def __init__(self, head_size):
    super().__init__()
    self.key = nn.Linear(n_embd, head_size, bias=False)
    self.query = nn.Linear(n_embd, head_size, bias=False)
    self.value = nn.Linear(n_embd, head_size, bias=False)
    self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

    self.dropout = nn.Dropout(dropout)

  def forward(self, x):
    B, T, C = x.shape
    k = self.key(x)
    q = self.query(x)
    v = self.value(x)

    wei = q @ k.transpose(-2,-1)*C**-0.5 # (B, T, C) @ (B, C, T) ---> (B, T, T)
    wei = wei.masked_fill(self.tril[:T, :T]==0, float('-inf'))
    wei = F.softmax(wei, dim=-1)

    v = self.value(x)
    out = wei @ v

    return out

class MultiHeadAttention(nn.Module):

  def __init__(self, num_heads, head_size):
    super().__init__()
    self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
    self.proj = nn.Linear(n_embd,n_embd)
    self.dropout = nn.Dropout(dropout)
  def forward(self, x):
    return self.dropout(self.proj(torch.cat([h(x) for h in self.heads], dim=-1)))

class FeedForward(nn.Module):
  """computation after attention"""
  def __init__(self, n_embd):
    super().__init__()
    self.net = nn.Sequential(nn.Linear(n_embd, 4*n_embd),
                             nn.ReLU(),
                             nn.Linear(4*n_embd, n_embd),
                             nn.Dropout(dropout),
                             )

  def forward(self, x):
    return self.net(x)

class Block(nn.Module):
  """Transformer block"""
  def __init__(self, n_embd, n_head):
    super().__init__()
    head_size = n_embd//n_head
    self.sa_head = MultiHeadAttention(n_head, head_size)
    self.ffwd = FeedForward(n_embd)
    self.ln1 = nn.LayerNorm(n_embd)
    self.ln2 = nn.LayerNorm(n_embd)

  def forward(self, x):
    x = x + self.sa_head(self.ln1(x))
    x = x + self.ffwd(self.ln2(x))
    return x

class BigramLanguageModel(nn.Module):

  def __init__(self):
    super().__init__()
    self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
    self.position_embedding_table = nn.Embedding(block_size, n_embd)
    self.blocks = nn.Sequential(*[Block(n_embd, n_head=4) for _ in range(n_layer)])
    self.ln_f = nn.LayerNorm(n_embd)
    self.lm_head = nn.Linear(n_embd, vocab_size)

  def forward(self, idx, targets=None):
    B, T = idx.shape
    # print('B:', B, 'T:', T)
    tok_emb = self.token_embedding_table(idx) # B, T, n_embd
    pos_emb = self.position_embedding_table(torch.arange(T)) # T, n_embd
    x = tok_emb + pos_emb
    x = self.blocks(x)
    x = self.ln_f(x)
    logits = self.lm_head(x) # B, T, C


    if targets is None:
      loss= None
    else:
      B, T, C = logits.shape
      logits = logits.view(B*T, C)
      targets = targets.view(B*T)
      loss = F.cross_entropy(logits, targets)

    return logits, loss

  def generate(self, idx, max_new_tokens):
    # idx is (B, T) array of indices in the current context
    for _ in range(max_new_tokens):
      # get the predictions
      idx_context = idx[:,-block_size:]
      logits, loss = self(idx_context)
      # last time step
      logits = logits[:,-1,:]

      probs = F.softmax(logits, dim=-1)

      idx_next = torch.multinomial(probs, num_samples=1)

      idx = torch.cat((idx, idx_next), dim=1)
    return idx


In [ ]:
def get_batch(split):
  data = train_data if split == 'train' else torch._validate_sparse_compressed_tensor_args
  ix  = torch.randint(len(data) - block_size, (batch_size, ))
  # print(ix)
  x = torch.stack([data[i:i+block_size] for i in ix])
  y = torch.stack([data[i+1:i+block_size+1] for i in ix])
  return x, y

xb, yb =  get_batch('train')

# print('inputs:')
# print(xb.shape)
# print(xb)
# print('targets:')
# print(yb.shape)
# print(yb)

# for b in range(batch_size):
#   for t in range(block_size):
#     context = xb[b,:t+1]
#     target = yb[b,t]
#     print(f"Input:{context.tolist()}, target: {target}")

m =  BigramLanguageModel()
logits, loss= m(xb, yb)
print(logits.shape)
print(loss)




print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=100)[0].tolist()))

torch.Size([256, 65])
tensor(4.4250, grad_fn=<NllLossBackward0>)

nl
ITw;mNCTTqfvEHT;bZEigs?LEEOk?NGztIOnFB'Gw?F
,O3QYfIBahanzEnrRkNABfcEXRPhEnjNUNZOMFl'j oQfaa;Gf?NM


In [ ]:
optimizer = torch.optim.AdamW(m.parameters(), lr=learning_rate)

In [ ]:

for steps in range(max_iters):
  xb, yb = get_batch('train')

  logits, loss = m(xb, yb)
  optimizer.zero_grad(set_to_none=True)
  loss.backward()
  optimizer.step()

print(loss.item())

2.0635480880737305


In [ ]:
print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=500)[0].tolist()))


DOBROKE:
YLa.

DUKE---
Yeath matizentersefter,
Bunssiold, I wellastadves fie of therenged you silennow

Fries there wrees pathrace
As to firks'dper; and it prand it sposter bestent's her; ads daked nembeeash Coning ay mar dessen faJifell'd you.

DORWARIUS:
Yow chan were mortherempon tell the pakesing, my reouch so? an with hee him lie;
Nyou with shall feest the the hestristy:
Leatiuse.

NCESIO:
Nay:
My meeneasing imall I core the peak; infolde
Dids a to in thungelm it is fatir clord the the sope


# Self-Attention

In [ ]:
torch.manual_seed(1337)

B, T, C = 4, 8, 2 # batch, time, channels

x = torch.randn(B, T, C)
x.shape

head_size = 16
key = nn.Linear(C, head_size, bias=False)
query = nn.Linear(C, head_size, bias=False)
value = nn.Linear(C, head_size, bias = False)

k = key(x)
q = query(x)
v = value(x)

wei = q @ k.transpose(-2, -1)*head_size**-0.5 # (B, T, 16) @ (B, 16, T) --> (B, T, T)



In [ ]:
# xbow = torch.zeros((B,T,C))
# #version1
# for b in range(B):
#   for t in range(T):
#     xprev = x[b, :t+1] # T, C
#     xbow[b, t] = torch.mean(xprev, 0) # C

# # xbow

# #version2
# wei = torch.ones(T, T)
# wei = torch.tril(wei)
# wei = wei / wei.sum(1, keepdim=True)
# xbow2 = wei @ x # (B, T, T) @ (B, T, C) ---> (B, T, C)

# print(torch.allclose(xbow, xbow2))
# wei

In [ ]:
#version3
tril = torch.tril(torch.ones(T,T))
#wei = torch.zeros((T,T))
wei = wei.masked_fill(tril == 0, float('-inf')) # Only in decoder blocks, delete in encoder blocks where all nodes talk to each other
wei = F.softmax(wei, dim=-1)
out = wei @ v
# print(torch.allclose(xbow,xbow3))
out.shape

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5299, 0.4701, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3275, 0.2903, 0.3822, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2270, 0.2091, 0.2591, 0.3048, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2007, 0.2208, 0.1976, 0.1959, 0.1851, 0.0000, 0.0000, 0.0000],
        [0.1447, 0.1548, 0.1403, 0.1386, 0.1440, 0.2776, 0.0000, 0.0000],
        [0.1424, 0.1371, 0.1473, 0.1517, 0.1465, 0.1323, 0.1427, 0.0000],
        [0.1093, 0.1192, 0.1078, 0.1074, 0.1078, 0.2209, 0.1093, 0.1184]],
       grad_fn=<SelectBackward0>)

In [ ]:
torch.manual_seed(42)

a = torch.ones(3, 3)
a = torch.tril(a)
print('a =','\n',a)
a = a / torch.sum(a, 1, keepdim=True)
b =  torch.randint(0, 10, (3, 2)).float()

c = a @ b

print('a =')
print(a)

print('b =')
print(b)

print('c =')
print(c)

a = 
 tensor([[1., 0., 0.],
        [1., 1., 0.],
        [1., 1., 1.]])
a =
tensor([[1.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000],
        [0.3333, 0.3333, 0.3333]])
b =
tensor([[2., 7.],
        [6., 4.],
        [6., 5.]])
c =
tensor([[2.0000, 7.0000],
        [4.0000, 5.5000],
        [4.6667, 5.3333]])
